In [5]:
import pygame
import random

## Tiles

In [14]:
# --- Config ---
WIDTH, HEIGHT = 800, 600
TILE_SIZE = 40
GRID_COLS = WIDTH // TILE_SIZE
GRID_ROWS = HEIGHT // TILE_SIZE
ELEVATION_COLORS = [(100, 100, 100), (120, 120, 120), (150, 150, 150), (180, 180, 180), (220, 220, 220)]

# --- Init ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 24)

# --- Generate Grid with Elevation ---
grid = [[random.randint(0, 4) for _ in range(GRID_COLS)] for _ in range(GRID_ROWS)]

def draw_grid():
    for y in range(GRID_ROWS):
        for x in range(GRID_COLS):
            elevation = grid[y][x]
            color = ELEVATION_COLORS[elevation]
            rect = pygame.Rect(x * TILE_SIZE, y * TILE_SIZE, TILE_SIZE, TILE_SIZE)
            pygame.draw.rect(screen, color, rect)
            pygame.draw.rect(screen, (50, 50, 50), rect, 1)

def draw_hover_info(mouse_pos):
    grid_x = mouse_pos[0] // TILE_SIZE
    grid_y = mouse_pos[1] // TILE_SIZE
    if 0 <= grid_x < GRID_COLS and 0 <= grid_y < GRID_ROWS:
        elevation = grid[grid_y][grid_x]
        text = font.render(f"Tile ({grid_x}, {grid_y}) Elevation: {elevation}", True, (255, 255, 255))
        screen.blit(text, (10, HEIGHT - 30))
        pygame.draw.rect(screen, (255, 0, 0), (grid_x * TILE_SIZE, grid_y * TILE_SIZE, TILE_SIZE, TILE_SIZE), 2)

# --- Main Loop ---
running = True
while running:
    screen.fill((30, 30, 30))
    draw_grid()
    draw_hover_info(pygame.mouse.get_pos())

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    pygame.display.flip()
    clock.tick(60)

pygame.quit()

## Tiles with Elelevation

In [12]:
# --- Config ---
WIDTH, HEIGHT = 800, 600
TILE_WIDTH = 64
TILE_HEIGHT = 32
GRID_COLS = 10
GRID_ROWS = 10
ELEVATION_SCALE = 10

# --- Init ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 24)

# --- Generate Grid with Elevation ---
grid = [[random.randint(0, 4) for _ in range(GRID_COLS)] for _ in range(GRID_ROWS)]

# --- Calculate Grid Offset for Centering ---
grid_pixel_width = (GRID_COLS + GRID_ROWS) * TILE_WIDTH // 2
grid_pixel_height = (GRID_COLS + GRID_ROWS) * TILE_HEIGHT // 2
offset_x = (WIDTH - grid_pixel_width) // 2
offset_y = (HEIGHT - grid_pixel_height) // 2

def grid_to_iso(x, y, elevation):
    screen_x = (x - y) * TILE_WIDTH // 2 + offset_x + grid_pixel_width // 2
    screen_y = (x + y) * TILE_HEIGHT // 2 - elevation * ELEVATION_SCALE + offset_y
    return screen_x, screen_y

def draw_tile(x, y, elevation):
    sx, sy = grid_to_iso(x, y, elevation)
    points = [
        (sx, sy),
        (sx + TILE_WIDTH // 2, sy + TILE_HEIGHT // 2),
        (sx, sy + TILE_HEIGHT),
        (sx - TILE_WIDTH // 2, sy + TILE_HEIGHT // 2)
    ]
    color = (100 + elevation * 30, 100 + elevation * 30, 100 + elevation * 30)
    pygame.draw.polygon(screen, color, points)
    pygame.draw.polygon(screen, (50, 50, 50), points, 1)

def draw_grid():
    for y in range(GRID_ROWS):
        for x in range(GRID_COLS):
            elevation = grid[y][x]
            draw_tile(x, y, elevation)

def draw_hover_info(mouse_pos):
    mx, my = mouse_pos
    dx = mx - (offset_x + grid_pixel_width // 2)
    dy = my - offset_y

    grid_x = (dx // (TILE_WIDTH // 2) + dy // (TILE_HEIGHT // 2)) // 2
    grid_y = (dy // (TILE_HEIGHT // 2) - dx // (TILE_WIDTH // 2)) // 2

    if 0 <= grid_x < GRID_COLS and 0 <= grid_y < GRID_ROWS:
        elevation = grid[grid_y][grid_x]
        text = font.render(f"Tile ({grid_x}, {grid_y}) Elevation: {elevation}", True, (255, 255, 255))
        screen.blit(text, (10, HEIGHT - 30))
        sx, sy = grid_to_iso(grid_x, grid_y, elevation)
        pygame.draw.circle(screen, (255, 0, 0), (sx, sy + TILE_HEIGHT // 2), 5)

# --- Main Loop ---
running = True
while running:
    screen.fill((30, 30, 30))
    draw_grid()
    draw_hover_info(pygame.mouse.get_pos())

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    pygame.display.flip()
    clock.tick(60)

pygame.quit()

In [16]:
import pygame
import random

# --- Config ---
WIDTH, HEIGHT = 800, 600
TILE_WIDTH = 64
TILE_HEIGHT = 32
GRID_COLS = 20
GRID_ROWS = 20
ELEVATION_SCALE = 10
CAMERA_SPEED = 10

# --- Init ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 24)

# --- Generate Grid with Elevation ---
grid = [[random.randint(-2, 4) for _ in range(GRID_COLS)] for _ in range(GRID_ROWS)]

# --- Camera Offset ---
camera_x = 0
camera_y = 0

def grid_to_iso(x, y, elevation):
    screen_x = (x - y) * TILE_WIDTH // 2 - camera_x + WIDTH // 2
    screen_y = (x + y) * TILE_HEIGHT // 2 - elevation * ELEVATION_SCALE - camera_y + HEIGHT // 2
    return screen_x, screen_y

def draw_tile(x, y, elevation):
    sx, sy = grid_to_iso(x, y, elevation)
    points = [
        (sx, sy),
        (sx + TILE_WIDTH // 2, sy + TILE_HEIGHT // 2),
        (sx, sy + TILE_HEIGHT),
        (sx - TILE_WIDTH // 2, sy + TILE_HEIGHT // 2)
    ]
    if elevation < 0:
        color = (50, 80, 180) if elevation <= -2 else (80, 120, 200)  # Deep vs shallow water
    elif elevation == 1:
        color = (100, 180, 100)  # Normal grass
    elif elevation == 2:
        color = (160, 140, 80)   # Hill
    elif elevation >= 3:
        color = (180, 180, 180)  # Building
    else:
        color = (120, 100, 80)   # Dirt
    pygame.draw.polygon(screen, color, points)
    pygame.draw.polygon(screen, (50, 50, 50), points, 1)

def draw_grid():
    for y in range(GRID_ROWS):
        for x in range(GRID_COLS):
            elevation = grid[y][x]
            draw_tile(x, y, elevation)

def draw_hover_info(mouse_pos):
    mx, my = mouse_pos
    dx = mx - WIDTH // 2 + camera_x
    dy = my - HEIGHT // 2 + camera_y

    grid_x = (dx // (TILE_WIDTH // 2) + dy // (TILE_HEIGHT // 2)) // 2
    grid_y = (dy // (TILE_HEIGHT // 2) - dx // (TILE_WIDTH // 2)) // 2

    if 0 <= grid_x < GRID_COLS and 0 <= grid_y < GRID_ROWS:
        elevation = grid[grid_y][grid_x]
        text = font.render(f"Tile ({grid_x}, {grid_y}) Elevation: {elevation}", True, (255, 255, 255))
        screen.blit(text, (10, HEIGHT - 30))
        sx, sy = grid_to_iso(grid_x, grid_y, elevation)
        pygame.draw.circle(screen, (255, 0, 0), (sx, sy + TILE_HEIGHT // 2), 5)

# --- Main Loop ---
running = True
while running:
    screen.fill((30, 30, 30))
    draw_grid()
    draw_hover_info(pygame.mouse.get_pos())

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT] or keys[pygame.K_a]:
        camera_x -= CAMERA_SPEED
    if keys[pygame.K_RIGHT] or keys[pygame.K_d]:
        camera_x += CAMERA_SPEED
    if keys[pygame.K_UP] or keys[pygame.K_w]:
        camera_y -= CAMERA_SPEED
    if keys[pygame.K_DOWN] or keys[pygame.K_s]:
        camera_y += CAMERA_SPEED

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    pygame.display.flip()
    clock.tick(60)

pygame.quit()

In [17]:
import pygame
import random
import json

# --- Config ---
WIDTH, HEIGHT = 800, 600
TILE_WIDTH = 64
TILE_HEIGHT = 32
GRID_COLS = 20
GRID_ROWS = 20
ELEVATION_SCALE = 10
CAMERA_SPEED = 10

# --- Init ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 24)

# --- Terrain Types ---
terrain_order = ["grass", "water", "hill", "building"]
terrain_colors = {
    "grass": (100, 180, 100),
    "water": (80, 120, 200),
    "hill": (160, 140, 80),
    "building": (180, 180, 180)
}

# --- Grid Data ---
def create_empty_grid():
    return [[{"elev": 1, "type": "grass"} for _ in range(GRID_COLS)] for _ in range(GRID_ROWS)]

grid = create_empty_grid()

# --- Camera ---
camera_x = 0
camera_y = 0

# --- Editor Mode ---
editor_mode = True

# --- Grid to Isometric Projection ---
def grid_to_iso(x, y, elevation):
    screen_x = (x - y) * TILE_WIDTH // 2 - camera_x + WIDTH // 2
    screen_y = (x + y) * TILE_HEIGHT // 2 - elevation * ELEVATION_SCALE - camera_y + HEIGHT // 2
    return screen_x, screen_y

# --- Draw Tile ---
def draw_tile(x, y, tile):
    elevation = tile["elev"]
    terrain = tile["type"]
    sx, sy = grid_to_iso(x, y, elevation)

    color = terrain_colors.get(terrain, (120, 100, 80))
    points = [
        (sx, sy),
        (sx + TILE_WIDTH // 2, sy + TILE_HEIGHT // 2),
        (sx, sy + TILE_HEIGHT),
        (sx - TILE_WIDTH // 2, sy + TILE_HEIGHT // 2)
    ]
    pygame.draw.polygon(screen, color, points)
    pygame.draw.polygon(screen, (50, 50, 50), points, 1)

# --- Draw Grid ---
def draw_grid():
    for y in range(GRID_ROWS):
        for x in range(GRID_COLS):
            draw_tile(x, y, grid[y][x])

# --- Hover Info ---
def draw_hover_info(mouse_pos):
    mx, my = mouse_pos
    dx = mx - WIDTH // 2 + camera_x
    dy = my - HEIGHT // 2 + camera_y

    grid_x = (dx // (TILE_WIDTH // 2) + dy // (TILE_HEIGHT // 2)) // 2
    grid_y = (dy // (TILE_HEIGHT // 2) - dx // (TILE_WIDTH // 2)) // 2

    if 0 <= grid_x < GRID_COLS and 0 <= grid_y < GRID_ROWS:
        tile = grid[grid_y][grid_x]
        text = font.render(f"Tile ({grid_x}, {grid_y}) Elev: {tile['elev']} Type: {tile['type']}", True, (255, 255, 255))
        screen.blit(text, (10, HEIGHT - 30))
        sx, sy = grid_to_iso(grid_x, grid_y, tile["elev"])
        pygame.draw.circle(screen, (255, 0, 0), (sx, sy + TILE_HEIGHT // 2), 5)

# --- Save/Load ---
def save_stage(filename="stage.json"):
    with open(filename, "w") as f:
        json.dump(grid, f)

def load_stage(filename="stage.json"):
    global grid
    with open(filename, "r") as f:
        grid = json.load(f)

# --- Main Loop ---
running = True
while running:
    screen.fill((30, 30, 30))
    draw_grid()
    draw_hover_info(pygame.mouse.get_pos())

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT] or keys[pygame.K_a]:
        camera_x -= CAMERA_SPEED
    if keys[pygame.K_RIGHT] or keys[pygame.K_d]:
        camera_x += CAMERA_SPEED
    if keys[pygame.K_UP] or keys[pygame.K_w]:
        camera_y -= CAMERA_SPEED
    if keys[pygame.K_DOWN] or keys[pygame.K_s]:
        camera_y += CAMERA_SPEED

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # --- Editor Mouse Input ---
        if editor_mode and event.type == pygame.MOUSEBUTTONDOWN:
            mx, my = pygame.mouse.get_pos()
            dx = mx - WIDTH // 2 + camera_x
            dy = my - HEIGHT // 2 + camera_y

            grid_x = (dx // (TILE_WIDTH // 2) + dy // (TILE_HEIGHT // 2)) // 2
            grid_y = (dy // (TILE_HEIGHT // 2) - dx // (TILE_WIDTH // 2)) // 2

            if 0 <= grid_x < GRID_COLS and 0 <= grid_y < GRID_ROWS:
                tile = grid[grid_y][grid_x]
                if event.button == 1:  # Left click: raise elevation
                    tile["elev"] += 1
                elif event.button == 3:  # Right click: cycle terrain
                    current = tile["type"]
                    next_index = (terrain_order.index(current) + 1) % len(terrain_order)
                    tile["type"] = terrain_order[next_index]

        # --- Save/Load Keys ---
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_s:
                save_stage()
            elif event.key == pygame.K_l:
                load_stage()

    pygame.display.flip()
    clock.tick(60)

pygame.quit()